In [1]:

Langue_de_reconnaissance = "Francais"
import json
import glob
import requests
import os
import pandas as pd
!pip install spacy==3.2
import urllib
!pip install --upgrade wptools
import wptools
if Langue_de_reconnaissance == 'Anglais':
  !python -m spacy download en_core_web_lg
  a_importer = 'en_core_web_lg'
if Langue_de_reconnaissance == "Francais":
  !python -m spacy download fr_core_news_lg
  a_importer = 'fr_core_news_lg'


import spacy
nlp = spacy.load(a_importer)

        


Using TensorFlow backend.
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/fr_core_news_lg-3.2.0/fr_core_news_lg-3.2.0-py3-none-any.whl#egg=fr_core_news_lg==3.2.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.9/572.9 MB 15.0 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


Using TensorFlow backend.


In [4]:
def CallWikifier(text, lang='fr', threshold=1): ### Changer ici la langue et le seuil si nécessaire
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", ""),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    print(text)
    url = "http://www.wikifier.org/annotate-article"
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
      #  print(response)
    for annotation in response["annotations"]:
          #  print(annotation)
            global itemid
           # print(itemid)
            try:
                itemid = annotation["wikiDataItemId"]
                global lien
                lien = ''.join(['https://www.wikidata.org/wiki/', itemid])
            except KeyError as ko:
                pass


        
        
        
        
        
nlp.max_length = 150000000

## Récupération données XLS + conversion en DICT
file = '/Users/marinagiardinetti/Desktop/valois_xls/Aisne.xlsx'
nom = os.path.basename(file)
df = pd.ExcelFile(file).parse('Sheet1')
df.reset_index(inplace=True)
liste_pers = []
liste_lieux = []
dicti = df.to_dict('records')
new_dict = {}
for element in dicti: 
    for key, value in element.items():
        if key == 'index':   
            new_dict[value] = element            

for key, value in new_dict.items():
    try:
        for key2, value2 in value.items():
            if key2 == 'legende':
                doc = nlp(str(value2))
                liste_loc, liste_pers, liste_lienL, liste_lienP = ([]for i in range(4))
                for ent in doc.ents:
                    
                    if ent.label_ == 'LOC': 
                        liste_loc.append(ent.text)
                        new_dict[key]['lieux'] = liste_loc
                      #  print(element)
                        CallWikifier(text=ent.text)
                      #  print(lien)
                        liste_lienL.append(lien)
                        new_dict[key]['lien_lieux'] = liste_lienL
                        
                    if ent.label_ == 'PER': 
                        liste_pers.append(ent.text)
                        new_dict[key]['pers'] = liste_pers
                        print(liste_pers)
                        CallWikifier(text=ent.text)
                        liste_lienP.append(lien)
                        print(liste_lienP)
                        new_dict[key]['lien_pers'] = liste_lienP
    except (RuntimeError, NameError) as pl:
        continue

        
df2 = pd.DataFrame(new_dict)
df3 = df2.transpose()
nomf = ''.join(['/Users/marinagiardinetti/Downloads/',nom[:-5], '_EN.xlsx'])
df3.to_excel(nomf)



Soissons
Panorama
Soissons
Soissons
Panorama
Cathédrale
Saint-Jean-des-Vignes
Soissons
Panorama
Saint-Léger
Crouy
Soissons
Hôtel de ville
Saint-Jean-des-Vignes
Soissons
Soissons
Panorama
Saint-Jean-des-Vignes
Soissons
Panorama
Saint-Jean-des-Vignes
Soissons
Panorama
Saint-Jean-des-Vignes
Soissons
Panorama
Saint-Jean-des-Vignes
Soissons
Panorama
Saint-Jean-des-Vignes
Soissons
Panorama
Saint-Jean-des-Vignes
Soissons
Soissons
Soissons
Panorama
Soissons
Panorama
Soissons
Panorama
Soissons
Panorama
Soissons
Panorama
Soissons
Panorama
Soissons
Panorama
Soissons
Panorama
boulevard Jeanne d'Arc
Saint-Jean-des-Vignes
Soissons
Soissons
Soissons
Panorama
Soissons
Panorama
Saint-Jean-des-Vignes
Soissons
Panorama
Soissons
Soissons
Soissons
Pont-Neuf
Soissons
Faubourg Saint-Waast
mont Vauxrot
Soissons
l'Aisne
Soissons
l'Aisne
Français
Allemands
Soissons
pont des Anglais
Saint-Waast
Saint-Waast
Allemands
Soissons
pont des Anglais
Saint-Waast
Vauxrot
le Pont-Neuf
Soissons
pont des Anglais
Saint-Waast


['https://www.wikidata.org/wiki/Q1253294', 'https://www.wikidata.org/wiki/Q2268486']
Soissons
Rue des Graviers
Soissons
Rue des Graviers
Soissons
Rue Saint-Christophe
Maison
Soissons
Rue du Collège
Maisons
Soissons
Rue du Collège
Maisons
Soissons
Rue du Collège
Soissons
Rue du Collège
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Rue Gaudin
Soissons
Villers-Cotterets
Soissons
Château-Thierry
['Consignes']
Consignes
['https://www.wikidata.org/wiki/Q846484']
Soissons
Château-Thierry
['Consignes']
Consignes
['https://www.wikidata.org/wiki/Q846484']
Soissons
Rue Racine
Maison
Soissons
Rue Quinette
Soissons
Rue Matigny
Maisons
Soissons
Place de la Gare
Allemands
Soissons
Saint-Jean-des-Vignes
Soissons
Saint-Jean-des-Vignes
Soissons
Saint-Jean-des-Vignes
Soissons
Rue Saint-Jean
Saint-Jean-des-Vignes
Allemands
Soissons
Rue Saint-Jean
Saint-Jean-des-Vig

['https://www.wikidata.org/wiki/Q1253294', 'https://www.wikidata.org/wiki/Q43437', 'https://www.wikidata.org/wiki/Q104680', 'https://www.wikidata.org/wiki/Q1253294']
['Huard', 'madame Macherez', 'colonel Modelon', 'madame Huard', 'mademoiselle Sellier']
mademoiselle Sellier
['https://www.wikidata.org/wiki/Q1253294', 'https://www.wikidata.org/wiki/Q43437', 'https://www.wikidata.org/wiki/Q104680', 'https://www.wikidata.org/wiki/Q1253294', 'https://www.wikidata.org/wiki/Q2268486']
['Huard', 'madame Macherez', 'colonel Modelon', 'madame Huard', 'mademoiselle Sellier', "général d'Anselme"]
général d'Anselme
['https://www.wikidata.org/wiki/Q1253294', 'https://www.wikidata.org/wiki/Q43437', 'https://www.wikidata.org/wiki/Q104680', 'https://www.wikidata.org/wiki/Q1253294', 'https://www.wikidata.org/wiki/Q2268486', 'https://www.wikidata.org/wiki/Q1653358']
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
['Dessart']
Dessart
['https://www.wikidata.org/wiki/Q167152']
Soissons
Lycée
Les Orato

Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Hôtel de ville
Soissons
Hôtel de ville
Soissons
Soissons
Soissons
Douches
Soissons
Soissons
Tranchée
Soissons
Tranchée
Soissons
Tranchée
Soissons
Tranchée
Soissons
Tranchée
Quartier Saint-Waast
Soissons
Soissons
Soissons
Allemands
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Soissons
Église Saint-Pierre
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Église Saint-Pierre-au-Parvis
Soissons
Saint-Léger
Soissons
Saint-Léger
Soissons
Saint-Léger
Soissons
Saint-Léger
Croisill

Soissons
Soissons
L'Aisne
Saint-Wasst
Soissons
Faubourg Saint-Waast
Panorama
Soissons
Faubourg Saint-Waast
Panorama
Soissons
L'Aisne
Faubourg Saint-Waast
Allemands
Soissons
Faubourg Saint-Waast
Soissons
Faubourg Saint-Waast
Soissons
Faubourg Saint-Waast
Panorama
Soissons
Faubourg Saint-Waast
Panorama
Soissons
Faubourg Saint-Waast
Panorama
Soissons
Faubourg Saint-Waast
Panorama
pont Saint-Waast
Anglais
Soissons
L'Aisne
Saint-Waast
pont des Anglais
Saint-Waast
Allemands
Soissons
Faubourg Saint-Waast
Crouy
Soissons
Faubourg Saint-Waast
chatel Clamecin
Soissons
Faubourg Saint-Waast
Crouy
Soissons
L'Aisne
Saint-Waast
Soissons
Faubourg Saint-Waast
Soissons
Soissons
Maison
Soissons
Maison
Petite Poterne
Soissons
Maison
Gosses
Gosses
Soissons
Faubourg Saint-Waast
Gosses
['prince de']
prince de
['https://www.wikidata.org/wiki/Q2747456']
Soissons
Faubourg Saint-Waast
Boyau
Soissons
Faubourg Saint-Waast
Soissons
Faubourg Saint-Waast
Soissons
Faubourg Saint-Waast
Tranchée
['prince de']
prince de
[

...

Plateau des Cravettes
Canon allemand
Plateau des Cravettes
Canon allemand
Plateau des Cravettes
Canon allemand
Pontavert
Panorama
Pontavert
Pontavert
Maisons
Pontavert
Pontavert
Pontavert
Pontavert
Maison
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Maison
['Napoléon']
Napoléon
['https://www.wikidata.org/wiki/Q517']
Pontavert
Maison
['Napoléon']
Napoléon
['https://www.wikidata.org/wiki/Q517']
Pontavert
Pontavert
Maison
Pontavert
Boyau de Denain
Pontavert
Pontavert
Pontavert
Pontavert
Maison
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Pontavert
Château d'Hédouville
Pontavert
['général Duplessis']
général Duplessis
['https://www.wikidata.org/wiki/Q3041434']
Pontavert
['général Duplessis']
général Duplessis
['https://www.wikidata.org/wiki/Q3041434']
Pontavert
Pontavert
Pontavert
Pontavert
Pont
Pontavert
Canal latéral à l'Aisne
Pontavert
l'Aisne
Pontavert
Canal

['https://www.wikidata.org/wiki/Q388452', 'https://www.wikidata.org/wiki/Q388452', 'https://www.wikidata.org/wiki/Q388452', 'https://www.wikidata.org/wiki/Q43437', 'https://www.wikidata.org/wiki/Q43437', 'https://www.wikidata.org/wiki/Q4167438', 'https://www.wikidata.org/wiki/Q1661326']
['Messieurs Lachaume', 'Chevalier', 'madame Puaux', 'madame Dennis', 'capitaine Puaux', 'madame Doherty', 'madame Sifton', 'Jusserand']
Jusserand
['https://www.wikidata.org/wiki/Q388452', 'https://www.wikidata.org/wiki/Q388452', 'https://www.wikidata.org/wiki/Q388452', 'https://www.wikidata.org/wiki/Q43437', 'https://www.wikidata.org/wiki/Q43437', 'https://www.wikidata.org/wiki/Q4167438', 'https://www.wikidata.org/wiki/Q1661326', 'https://www.wikidata.org/wiki/Q1685608']
['Messieurs Lachaume', 'Chevalier', 'madame Puaux', 'madame Dennis', 'capitaine Puaux', 'madame Doherty', 'madame Sifton', 'Jusserand', 'Catta Preta']
Catta Preta
['https://www.wikidata.org/wiki/Q388452', 'https://www.wikidata.org/wiki/

Château-Thierry
['Poincaré']
Poincaré
['https://www.wikidata.org/wiki/Q191974']
['Poincaré', 'Degoutte']
Degoutte
['https://www.wikidata.org/wiki/Q191974', 'https://www.wikidata.org/wiki/Q3167633']
['Poincaré', 'Degoutte', 'Duparge']
Duparge
['https://www.wikidata.org/wiki/Q191974', 'https://www.wikidata.org/wiki/Q3167633', 'https://www.wikidata.org/wiki/Q3167633']
Aisne
Château-Thierry
Château-Thierry
['général Degoutte']
général Degoutte
['https://www.wikidata.org/wiki/Q3167633']
['général Degoutte', 'Poincaré']
Poincaré
['https://www.wikidata.org/wiki/Q3167633', 'https://www.wikidata.org/wiki/Q191974']
Château-Thierry
Château-Thierry
Château-Thierry
Château-Thierry
Château-Thierry
Château-Thierry
['Lebrun']
Lebrun
['https://www.wikidata.org/wiki/Q194145']
Blocus
Château-Thierry
Aisne
Château-Thierry
['Président Poincaré']
Président Poincaré
['https://www.wikidata.org/wiki/Q191974']
['Président Poincaré', 'Degoutte']
Degoutte
['https://www.wikidata.org/wiki/Q191974', 'https://www.wik

Château-Thierry
Vaux
Château-Thierry
Vaux
Château-Thierry
Vaux
Château-Thierry
Vaux
Château-Thierry
Bois Belleau
Vaux
Château-Thierry
Bois Belleau
Vaux
Château-Thierry
Bois Belleau
Vaux
Château-Thierry
Bois Belleau
Vaux
Château-Thierry
Bois Belleau
Vaux
Château-Thierry
Bois Belleau
Vaux
Château-Thierry
Bois Belleau
Vaux
Château-Thierry
Vaux
Château-Thierry
Château-Thierry
Meaux
Vaux
Château-Thierry
Château-Thierry
Meaux
Vaux
Château-Thierry
Vaux
Château-Thierry
Vaux
Château-Thierry
Vaux
Château-Thierry
Vaux
Château-Thierry
Vaux
Château-Thierry
Bois de la Marette
Vaux
Vaux
Bois de la Marette
Vaux
Vaux
Bois de la Marette
Vaux
Vaux
Bois de la Marette
Vaux
Vaux
Bois de la Marette
Vaux
Américains
Bois de la Marette
Vaux
Français
Bouresches
Bouresches
de France
Bois Belleau
Bouresches
de France
Bois Belleau
Bouresches
de France
Bois Belleau
Bouresches
de France
Bois Belleau
Bouresches
de France
Belleau
Américains
Bois Belleau
Belleau
Allemands
Bois Belleau
Bois Belleau
Bois Belleau
Bois Bell

Bois du Chatelet
Brécy
bertha
Paris
['Clemenceau']
Clemenceau
['https://www.wikidata.org/wiki/Q171730']
['Clemenceau', 'Loucheur']
Loucheur
['https://www.wikidata.org/wiki/Q171730', 'https://www.wikidata.org/wiki/Q2788498']
Bois du Chatelet
Brécy
bertha
Paris
['Clemenceau']
Clemenceau
['https://www.wikidata.org/wiki/Q171730']
['Clemenceau', 'Loucheur']
Loucheur
['https://www.wikidata.org/wiki/Q171730', 'https://www.wikidata.org/wiki/Q2788498']
Bois du Chatelet
Brécy
bertha
Paris
Bois du Chatelet
Brécy
bertha
Paris
Bois du Chatelet
Brécy
bertha
Paris
Bois du Chatelet
Brécy
bertha
Paris
Bois du Chatelet
Brécy
bertha
Paris
Bois du Chatelet
Brécy
bertha
Paris
Bois du Chatelet
Brécy
bertha
Paris
Bois du Chatelet
Brécy
bertha
Paris
porte cochère
Maison
Verdilly
['colonel Martin']
colonel Martin
['https://www.wikidata.org/wiki/Q104680']
Verdilly
['colonel Martin']
colonel Martin
['https://www.wikidata.org/wiki/Q104680']
Verdilly
Verdilly
Verdilly
Verdilly
Verdilly
Verdilly
Avion allemand
Créz